In [1]:
import torch
from config import args, device
from predict.predict_utils import load_model_trained, predict_single_points
import torchvision
from torchvision.transforms import transforms, InterpolationMode
from utils.my_dataset import FibersDataset, WoodsDataset
from utils import create_circular_mask
from matplotlib import pyplot as plt
import numpy as np
torch.__version__

ImportError: libcupti.so.11.7: cannot open shared object file: No such file or directory

In [ ]:
def custom_config(args):
    args.img_size = 180
    args.dim_first = 2
    args.dim_second = 3
    args.dim_third = 4
    args.batch_size = 8
    args.margin_loss = 2.0
    args.border  = 15
    args.exported = False
    return args

In [ ]:
    args = custom_config(args)
    transform = transforms.Compose([
        transforms.Resize((args.img_size, args.img_size), interpolation=InterpolationMode.NEAREST),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    testset = torchvision.datasets.Flowers102(root='./data/datasets', split='test',
                                                download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=args.batch_size,
                                             shuffle=False, num_workers=2)

In [ ]:
from training import KeyEqGroup, KeyPointsSelection, remove_borders, random_augmentation, shifted_batch_tensor
from utils import load_model, imshow, imshow2, imshow3,NMSHead,get_features


MODEL_PATH = "./data/models/model_flowers_ssim.pt"

model = load_model_trained(args,device,path=MODEL_PATH)

In [ ]:
iterator=iter(testloader)
batch = next(iterator)
img_batch, labels =batch
print(len(img_batch),labels) #4

In [ ]:
batch_result,orien1 = predict_single_points(model,batch)

In [ ]:
print('QTD batch ',len(batch_result))
for i,b in enumerate(batch_result):
    print('batch ',i,' QTD features ', len(b))

In [ ]:
img_batch.shape

In [ ]:
_kp1, _orie1 = model(img_batch.to(device))

In [ ]:
for i,detect in enumerate(batch_result):
    print('batch ',i,' QTD features ', len(detect))
    for j,(box,img) in enumerate(detect):
        plt.imshow(img)
        plt.show()
    break

In [ ]:
from e2cnn import gspaces, nn
r2_act = gspaces.Rot2dOnR2(N=args.group_size)
feat_type_ori_est = nn.FieldType(r2_act, [r2_act.regular_repr])
pool_max = nn.PointwiseMaxPoolAntialiased(feat_type_ori_est,kernel_size=2)

In [ ]:
feat_type_ori_est

In [ ]:
img_temp = batch_result[0][0][1]
_orie1.shape

In [ ]:
g_size = _orie1.shape[1]
print(_orie1.shape)
filtro = _orie1[0,:,25:50,100:120]
print(filtro.shape)
for i in range(g_size):
    plt.imshow(filtro[i].cpu().detach())
    plt.show()